In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time
print("Package loaded!!")

Package loaded!!


In [2]:
_data_dir = "./mnist/"

mnist = input_data.read_data_sets(_data_dir, one_hot=True)
print("Data loaded!!")

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Data loaded!!


In [3]:
img_test = mnist.test.images
label_test = mnist.test.labels

NUM_CLASSES = 10
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE  # 784
NUM_HIDDEN1_NODE = 256
NUM_HIDDEN2_NODE = 128

stddev = 0.1

x = tf.placeholder("float", [None, IMAGE_PIXELS])
y = tf.placeholder("float", [None, NUM_CLASSES])

In [4]:
def xavier_init(shape, distribution_type="uniform"):
    input_size = shape[0]
    output_size = shape[1]
    n = input_size + output_size
    if distribution_type == "normal":
        stddev = tf.sqrt(2.0 / n)
        return tf.random_normal([input_size, output_size], stddev=stddev)
    else:
        interval = tf.sqrt(6.0 / n)
        return tf.random_uniform([input_size, output_size], minval=-interval, maxval=interval)

In [5]:
def linear(data_in, output_size, name="linear"):
    with tf.name_scope(name):
        input_size = data_in.get_shape().as_list()[-1]
        # weights = tf.Variable(xavier_init([input_size, output_size], "normal"), name="weights")
        weights = tf.Variable(xavier_init([input_size, output_size], "uniform"), name="weights")
        biases = tf.Variable(tf.zeros(output_size), name="biases")
        logits = tf.nn.xw_plus_b(data_in, weights, biases, name="logits")
        activation = tf.nn.sigmoid(logits, name="act")
        return logits, activation


def inference_mlp1(data_in, num_hidden_node):
    _, h1 = linear(data_in, num_hidden_node, name="linear1")
    logits, _ = linear(h1, NUM_CLASSES, name="linear2")
    return logits


def inference_mlp2(data_in, num_hidden1_node, num_hidden2_node):
    _, h1 = linear(data_in, num_hidden1_node, name="linear1")
    _, h2 = linear(h1, num_hidden2_node, name="linear2")
    logits, _ = linear(h2, NUM_CLASSES, name="linear3")
    return logits


def cost(logits, labels):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))


def training(loss, learning_rate):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train = optimizer.minimize(loss, global_step=global_step)
    return train


def evaluation(logits, labels):
    pred = tf.equal(tf.argmax(tf.nn.softmax(logits), 1), tf.argmax(labels, 1))
    accr = tf.reduce_mean(tf.cast(pred, "float"))
    return accr

In [6]:
learning_rate = 0.001

#logits = inference_mlp1(x, NUM_HIDDEN1_NODE)
logits = inference_mlp2(x, NUM_HIDDEN1_NODE, NUM_HIDDEN2_NODE)
loss = cost(logits, y)
train = training(loss, learning_rate=learning_rate)
eval_correct = evaluation(logits, y)

init = tf.initialize_all_variables()

n_samples = mnist.train.num_examples
batch_size = 200
total_batch = int(n_samples / batch_size)
start = time.time()
total_epoch = 50

# SESSION
with tf.Session() as sess:
    sess.run(init)
    # MINI-BATCH LEARNING
    for epoch in range(total_epoch + 1):
        avg_cost = 0.
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feed = {x: batch_xs, y: batch_ys}
            sess.run(train, feed)
            avg_cost += sess.run(loss, feed) / batch_size

        # DISPLAY
        if epoch % 5 == 0:
            feeds_train = {x: batch_xs, y: batch_ys}
            feeds_test = {x: img_test, y: label_test}
            train_acc = sess.run(eval_correct, feeds_train)
            test_acc = sess.run(eval_correct, feeds_test)
            print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f"
                   % (epoch, total_epoch, avg_cost, train_acc, test_acc))
print ("DONE")
print time.time() - start

Epoch: 000/050 cost: 1.041551063 train_acc: 0.955 test_acc: 0.913
Epoch: 005/050 cost: 0.169847944 train_acc: 0.960 test_acc: 0.962
Epoch: 010/050 cost: 0.076726905 train_acc: 0.995 test_acc: 0.975
Epoch: 015/050 cost: 0.035171795 train_acc: 0.990 test_acc: 0.979
Epoch: 020/050 cost: 0.015144050 train_acc: 0.990 test_acc: 0.978
Epoch: 025/050 cost: 0.006092057 train_acc: 1.000 test_acc: 0.981
Epoch: 030/050 cost: 0.002252830 train_acc: 1.000 test_acc: 0.980
Epoch: 035/050 cost: 0.001006011 train_acc: 1.000 test_acc: 0.981
Epoch: 040/050 cost: 0.000511293 train_acc: 1.000 test_acc: 0.980
Epoch: 045/050 cost: 0.000279982 train_acc: 1.000 test_acc: 0.981
Epoch: 050/050 cost: 0.000175718 train_acc: 1.000 test_acc: 0.981
DONE
51.0766100883
